In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper=parameters

input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())


In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Fully connected neural network with one hidden layer

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], step [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)

with torch.no_grads():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/5], step [100/600], loss:0.3478
Epoch [1/5], step [200/600], loss:0.2426
Epoch [1/5], step [300/600], loss:0.1123
Epoch [1/5], step [400/600], loss:0.2898
Epoch [1/5], step [500/600], loss:0.2645
Epoch [1/5], step [600/600], loss:0.2519
Epoch [2/5], step [100/600], loss:0.0807
Epoch [2/5], step [200/600], loss:0.2149
Epoch [2/5], step [300/600], loss:0.1247
Epoch [2/5], step [400/600], loss:0.1216
Epoch [2/5], step [500/600], loss:0.0511
Epoch [2/5], step [600/600], loss:0.0970
Epoch [3/5], step [100/600], loss:0.0717
Epoch [3/5], step [200/600], loss:0.0661
Epoch [3/5], step [300/600], loss:0.1212
Epoch [3/5], step [400/600], loss:0.1307
Epoch [3/5], step [500/600], loss:0.0843
Epoch [3/5], step [600/600], loss:0.0596
Epoch [4/5], step [100/600], loss:0.0500
Epoch [4/5], step [200/600], loss:0.0548
Epoch [4/5], step [300/600], loss:0.0847
Epoch [4/5], step [400/600], loss:0.0574
Epoch [4/5], step [500/600], loss:0.1209
Epoch [4/5], step [600/600], loss:0.0470
Epoch [5/5], ste

AttributeError: module 'torch' has no attribute 'no_grads'

In [ ]:
torch.save(model.state_dict(), 'modelfnn.ckpt')